In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Desktop\\xReayDetection\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Desktop\\xReayDetection'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/entbappy/Kidney-Disease-Classification-MLflow-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="entbappy"
os.environ["MLFLOW_TRACKING_PASSWORD"]="6824692c47a369aa6f9eac5b10041d5c8edbcef0"

In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [10]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [11]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/entbappy/Kidney-Disease-Classification-MLflow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [14]:
pip install mlflow


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.12.2 which is incompatible.



  Using cached importlib_metadata-8.4.0-py3-none-any.whl.metadata (4.7 kB)
   ---------------------------------------- 0.0/26.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/26.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/26.7 MB ? eta -:--:--
   ---------------------------------------- 0.3/26.7 MB ? eta -:--:--
   ---------------------------------------- 0.3/26.7 MB ? eta -:--:--
    --------------------------------------- 0.5/26.7 MB 599.9 kB/s eta 0:00:44
    --------------------------------------- 0.5/26.7 MB 599.9 kB/s eta 0:00:44
   - -------------------------------------- 0.8/26.7 MB 599.2 kB/s eta 0:00:44
   - -------------------------------------- 0.8/26.7 MB 599.2 kB/s eta 0:00:44
   - -------------------------------------- 1.0/26.7 MB 629.1 kB/s eta 0:00:41
   - -------------------------------------- 1.3/26.7 MB 651.7 kB/s eta 0:00:39
   - -------------------------------------- 1.3/26.7 MB 651.7 kB/s eta 0:00:39
   -- ---------------

In [15]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [16]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [17]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

Directory created or already exists: artifacts
Found 3733 images belonging to 1 classes.
234/234 [==============================] - 993s 4s/step - loss: 33.0572 - accuracy: 1.0000


2024/09/18 21:26:05 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\hp\AppData\Local\Temp\tmpy1e18di7\model\data\model\assets


2024/09/18 21:26:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/09/18 21:27:41 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 47
Created version '47' of model 'VGG16Model'.
2024/09/18 21:27:42 INFO mlflow.tracking._tracking_service.client: 🏃 View run crawling-moose-368 at: https://dagshub.com/entbappy/Kidney-Disease-Classification-MLflow-DVC.mlflow/#/experiments/0/runs/628920427f09431e910cef62fc2ad864.
2024/09/18 21:27:42 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/entbappy/Kidney-Disease-Classification-MLflow-DVC.mlflow/#/experiments/0.
